## Homework 5 可解释模型

In [ ]:
import torch
import torch.nn as nn 

In [ ]:
# ------------------------------- 基本变量 ---------------------------------
checkpoint_path = ""
train_data_path = ""


In [ ]:

# ------------------------------- 被解释模型定义 ---------------------------------

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # 64 * 128 * 128
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # 64 * 64 * 64

            nn.Conv2d(64, 128, 3, 1, 1),  # 128 * 64 * 64
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),   # 128 * 32 * 32

            nn.Conv2d(128, 256, 3, 1, 1),  # 256,32,32
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),    # 256, 16,16

            nn.Conv2d(256, 512, 3, 1, 1),  # 512,8,8
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),    # 512,8,8

            nn.Conv2d(512, 512, 3, 1, 1),  # 512,8,8
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)  # 512, 4,4
        )
        self.fn = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


In [ ]:

# ------------------------------- 加载模型 ---------------------------------
device = "cpu"
model = Classifier()
model = model.to(device)
checkpoint = torch.load(checkpoint_path)
# 加载模型参数
model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:

# ------------------------------- 定义数据集 ---------------------------------

class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None) -> None:
        super().__init__()
        self.x = x
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X
            
    def getbatch(self, indices):
        images = []
        labels = []
        for index in indices:
          image, label = self.__getitem__(index)
          images.append(image)
          labels.append(label)
        return torch.stack(images), torch.tensor(labels)


## Saliency Map

一般情况下，我们改变Model parameter 来拟合 image 与Label ，所以loss 在计算 backward时，我们只在乎loss 对 model parameter 的偏微分。

但是从数值上看，image 本身也是一个连续tensor，所以我们可以计算loss 对 input image的偏微分。这个偏微分表示，在model 和parameter 固定不变的情况下，改变image的某个像素 pixel value 会对loss 产生什么样的影响。

习惯上，把loss变化剧烈程度解释为当前pixel的重要度。

In [ ]:
def normalize(image):
    return (image-image.min())/(image.max() - image.min())

def compute_saliency_maps(x,y,model):
    model.eval()
    X = x.to(device)
    Y = y.to(device)

    # 使得X具有梯度，因为我们要计算 loss 对 input部分的微分
    X.requires_grad_()

    Y_hat = model(X)
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(Y_hat,Y)
    # 计算 loss 对 X 的偏微分
    loss.backward()

    salienies = X.grad.abs().detach().cpu()
    # 不同图片的Gradient 可能有很大落差，第一张图片Gradient 在100-10000 之间，而第二张图片Gradient 在0.001 - 0.01 之间，这就造成如果我们使用同样的色阶画图，第一张图片就会非常亮，而第二张图片非常暗。所以对每张saliency 做norma
    salienies = torch.stack([normalize(item) for item in salienies])
    return salienies

    

In [ ]:
img_indices = [83, 100, 750,500]
# 从数据集中获取 Input 和 label
images, labels = train_set.getbatch(img_indices)
saliencies = compute_saliency_maps(images,labels,model)

# 利用 matplatlib 绘图
ig, axs = plt.subplots(2, len(img_indices), figsize=(15, 8)) 
for row,target in enumerate([images,saliencies]):
    for column ,img in enumerate(target):
        axs[row][column].imshow(img.permute(1,2,0),numpy())
        # image tensor dimension是 channel,height,width
        # 而 matplatlib 需要的形状为：height,width,channels
plt.show()    


## Filter Explanation
如果想知道某个 filter 到底有什么作用，我们需要做两件事情
* Filter activation： 选几张图片，看看图片中有哪些位置会 activate 当前 filter
* Filter visualization: 怎样的图片，可以最大程度的 activate 当前  Filter

In [ ]:
layer_activations = None
def filter_explanation(x,model,filter_id,iteration = 100,lr = 1):
    model.eval()
    def hook(modek,input,output):
        global layer_activations
        layer_activations = output
    
    model.cnn[cnn_id].register_forsward_hook(hook)
    # 告诉pytorch 当 forward 经过 filter_id 时，先呼叫hook后，才可以继续 forward
    
    model(x.to(device))

    filter_activations = layer_activations[:,filter_id,:,:].detach().cpu()

    objective.backward()
    optimizer.step()

filter_visualization = x.detach().cpu().squeeze()[0]
  hook_handle.remove()


## Lime Explanation


In [ ]:
def predict(input):
    model.eval()
    # batch channel height width
    input = torch.FloatTensor(input).permute(0,3,1,2)
    output = model(input.to(device))
    return output.detach().cpu().numpy()



def segmentation(input):
    # 利用 skimage 提供的 segmentation 将图片分成100分
    return slic(input,n_segments = 100,compactness=1,sigma=1)

In [ ]:
                  
img_indices = [83, 4218, 4707, 8598]
images, labels = train_set.getbatch(img_indices)
fig, axs = plt.subplots(1, 4, figsize=(15, 8))                                                                                                                                                                 
np.random.seed(16)                                                                                                                                                       
# 讓實驗 reproducible
for idx, (image, label) in enumerate(zip(images.permute(0, 2, 3, 1).numpy(), labels)):                                                                                                                                             
    x = image.astype(np.double)
    # lime 這個套件要吃 numpy array

    explainer = lime_image.LimeImageExplainer()                                                                                                                              
    explaination = explainer.explain_instance(image=x, classifier_fn=predict, segmentation_fn=segmentation)
    # 基本上只要提供給 lime explainer 兩個關鍵的 function，事情就結束了
    # classifier_fn 定義圖片如何經過 model 得到 prediction
    # segmentation_fn 定義如何把圖片做 segmentation

    lime_img, mask = explaination.get_image_and_mask(                                                                                                                         
                                label=label.item(),                                                                                                                           
                                positive_only=False,                                                                                                                         
                                hide_rest=False,                                                                                                                             
                                num_features=11,                                                                                                                              
                                min_weight=0.05                                                                                                                              
                            )
    # 把 explainer 解釋的結果轉成圖片
    
    axs[idx].imshow(lime_img)

plt.show()
# 從以下前三章圖可以看到，model 有認出食物的位置，並以該位置為主要的判斷依據
# 唯一例外是第四張圖，看起來 model 似乎比較喜歡直接去認「碗」的形狀，來判斷該圖中屬於 soup 這個 class
# 至於碗中的內容物被標成紅色，代表「單看碗中」的東西反而有礙辨認。
# 當 model 只看碗中黃色的一坨圓形，而沒看到「碗」時，可能就會覺得是其他黃色圓形的食物。